In [1]:
import nltk
from nltk.corpus import movie_reviews

from sklearn.model_selection import train_test_split

import numpy as np
from pathlib import Path

import torch

import evaluate

from transformers import BertTokenizer, BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments

In [2]:
result_path = str(Path.cwd().parent / 'exercisebook_large_data' / 'Transformers' / 'results')
result_path

'c:\\Coding\\Local\\exercisebook_large_data\\Transformers\\results'

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


In [4]:
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)

In [5]:
sentence1 = "What a beautiful day!"
sentence2 = "Nvidia Titan XP has 12GB of VRAM"
print(tokenizer.tokenize(sentence1), tokenizer.tokenize(sentence2))

['what', 'a', 'beautiful', 'day', '!'] ['n', '##vid', '##ia', 'titan', 'xp', 'has', '12', '##gb', 'of', 'vr', '##am']


In [6]:
inputs = tokenizer([sentence1, sentence2], padding=True)
print(inputs)

{'input_ids': [[101, 2054, 1037, 3376, 2154, 999, 102, 0, 0, 0, 0, 0, 0], [101, 1050, 17258, 2401, 16537, 26726, 2038, 2260, 18259, 1997, 27830, 3286, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


In [7]:
inputs = tokenizer(sentence1, sentence2, padding=True)
print(inputs)

{'input_ids': [101, 2054, 1037, 3376, 2154, 999, 102, 1050, 17258, 2401, 16537, 26726, 2038, 2260, 18259, 1997, 27830, 3286, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [8]:
fileids = movie_reviews.fileids()
reviews = [movie_reviews.raw(fileid) for fileid in fileids]
categories = [movie_reviews.categories(fileid)[0] for fileid in fileids] 

label_dict = {'pos':1, 'neg':0}
y = [label_dict[c] for c in categories]

In [9]:
X_train_val, X_test, y_train_val, y_test = train_test_split(reviews, y, test_size=0.2, random_state=7)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.3, random_state=0)

In [10]:
tokenizer = BertTokenizerFast.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)
model = model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# Trainer 클래스를 활용할 경우 이 단계에서 GPU로 미전송 → Trainer 클래스가 알아서 보내주므로
train_input = tokenizer(X_train, truncation=True, padding=True, return_tensors='pt')
val_input = tokenizer(X_val, truncation=True, padding=True, return_tensors='pt')
test_input = tokenizer(X_test, truncation=True, padding=True, return_tensors='pt')
print(list(train_input.keys()))

['input_ids', 'token_type_ids', 'attention_mask']


In [12]:
class OurDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)

In [13]:
train_dataset = OurDataset(train_input, y_train)
val_dataset = OurDataset(val_input, y_val)
test_dataset = OurDataset(test_input, y_test)

In [14]:
metric = evaluate.load('accuracy')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    pred = np.argmax(logits, axis=-1)
    return metric.compute(predictions=pred, references=labels)

In [15]:
training_args = TrainingArguments(
    output_dir=result_path,
    logging_dir='./logs',
    num_train_epochs=10,
    per_device_train_batch_size=16, per_device_eval_batch_size=16
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    compute_metrics=compute_metrics
)

In [16]:
trainer.train()

C:\Users\Admin\AppData\Local\Temp\ipykernel_6052\475304883.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}


Step,Training Loss
500,0.703500


C:\Users\Admin\AppData\Local\Temp\ipykernel_6052\475304883.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}


TrainOutput(global_step=700, training_loss=0.7026687622070312, metrics={'train_runtime': 241.1199, 'train_samples_per_second': 46.45, 'train_steps_per_second': 2.903, 'total_flos': 2946843820032000.0, 'train_loss': 0.7026687622070312, 'epoch': 10.0})

In [17]:
trainer.evaluate(eval_dataset=test_dataset)

C:\Users\Admin\AppData\Local\Temp\ipykernel_6052\475304883.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}


{'eval_loss': 0.6930966377258301,
 'eval_accuracy': 0.505,
 'eval_runtime': 2.9574,
 'eval_samples_per_second': 135.252,
 'eval_steps_per_second': 8.453,
 'epoch': 10.0}